# PaddleGAN练习-「多人版」蚂蚁呀嘿


PaddleGAN施魔法前：

<div align='left'>
  <img src='./WechatIMG12.png' width='600'/>
</div>

PaddleGAN施魔法后：

<div align='left'>
  <img src='./finalout.gif' width='600'/>
</div>



整体流程分为三步：
1. 将照片中的多人脸使用人脸检测模型S3FD框出并抠出
2. 对抠出的人脸用First Order Motion进行脸部表情迁移
3. 将迁移好的人脸放回对应的原位置

**注意，由于人脸检测后，需要确定位置将人脸框出来并抠出，如果人脸太近，会框到除了其他的脸，导致效果较差**
**所以尽量选取人脸间距较大的照片，同时，框的大小需要根据照片中人脸间距情况进行调节（参数--ratio）**

## 下载PaddleGAN

In [5]:
# 从github上克隆PaddleGAN代码
#!git clone https://github.com/PaddlePaddle/PaddleGAN
!git clone https://gitee.com/paddlepaddle/PaddleGAN.git
%cd PaddleGAN
!git checkout develop

fatal: destination path 'PaddleGAN' already exists and is not an empty directory.
/home/aistudio/PaddleGAN
Already on 'develop'
Your branch is up-to-date with 'origin/develop'.


In [6]:
# 安装所需安装包
!pip install -r requirements.txt
!pip install imageio-ffmpeg
%cd applications/

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 29.2MB 9.0MB/s eta 0:00:012
     |████████████████████████████████| 1.6MB 27.5MB/s eta 0:00:01
     |████████████████████████████████| 4.4MB 14.1MB/s eta 0:00:01
     |████████████████████████████████| 174kB 23.7MB/s eta 0:00:01
  Created wheel for librosa: filename=librosa-0.7.0-cp37-none-any.whl size=1598345 sha256=57bf654a04fef67fadcc37478320c7ddcbd4143721e5065f047cbe14c9c3c380
  Stored in directory: /home/aistudio/.cache/pip/wheels/81/7e/60/c27574fffbf2f28075dbf4b28c00d3fe3240fefb51d597932e
Successfully built librosa
  Found existing installation: librosa 0.7.2
    Uninstalling librosa-0.7.2:
      Successfully uninstalled librosa-0.7.2
Looking in indexes: https://mirror.baidu.com/pypi/simple/
/home/aistudio/PaddleGAN/applications


## 执行命令

上传自己准备的视频和图片，并在如下命令中的source_image参数和driving_video参数分别换成自己的图片和视频路径，然后运行如下命令，就可以完成动作表情迁移，程序运行成功后，会在ouput文件夹生成名为result.mp4的视频文件，该文件即为动作迁移后的视频。

同时，根据自己上传的照片中人脸的间距，

本项目中提供了原始图片和驱动视频供展示使用。具体的各参数使用说明如下
- driving_video: 驱动视频，视频中人物的表情动作作为待迁移的对象
- source_image: 原始图片，视频中人物的表情动作将迁移到该原始图片中的人物上
- relative: 指示程序中使用视频和图片中人物关键点的相对坐标还是绝对坐标，建议使用相对坐标，若使用绝对坐标，会导致迁移后人物扭曲变形
- adapt_scale: 根据关键点凸包自适应运动尺度
- ratio: 将框出来的人脸贴回原图时的区域占宽高的比例，默认为0.4，范围为【0.4，0.5】
- multi_person: 设置为多人脸转换




In [7]:
!export PYTHONPATH=$PYTHONPATH:/home/aistudio/PaddleGAN && python -u tools/first-order-demo.py  --driving_video ~/work/2.MP4  --source_image /home/aistudio/WechatIMG12.png --ratio 0.4 --relative --adapt_scale --multi_person

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
[08/03 18:00:31] ppgan INFO: Downloading vox-cpk.pdparams from https://paddlegan.bj.bcebos.com/applications/first_order_model/vox-cpk.pdparams to /home/aistudio/.cache/ppgan/vox-cpk.pdparams
100%|████████████████████████████████| 350931/350931 [00:04<00:00, 72712.11it/s]
W0803 18:00:36.523557   220 device_context.cc:362] Please NOTE: device: 0, GPU Comp

## 最后一步：使用moviepy为生成的视频加上音乐

In [8]:
# add audio
!pip install moviepy

Looking in indexes: https://mirror.baidu.com/pypi/simple/


In [9]:
#为生成的视频加上音乐
from moviepy.editor import *

videoclip_1 = VideoFileClip("/home/aistudio/work/2.MP4")
videoclip_2 = VideoFileClip("./output/result.mp4")

audio_1 = videoclip_1.audio

videoclip_3 = videoclip_2.set_audio(audio_1)
videoclip_3.write_videofile("./output/finalout.mp4", audio_codec="aac")

t:   0%|          | 0/223 [00:00<?, ?it/s, now=None]                

Moviepy - Building video ./output/finalout.mp4.
MoviePy - Writing audio in finaloutTEMP_MPY_wvf_snd.mp4
MoviePy - Done.
Moviepy - Writing video ./output/finalout.mp4



t:   9%|▊         | 19/223 [00:00<00:01, 147.83it/s, now=None]                                                              


Moviepy - Done !
Moviepy - video ready ./output/finalout.mp4


# 多人版的蚂蚁呀嘿完成！

至此，多人版本的蚂蚁呀嘿就制作完成啦~

**大家快来动手尝试吧！记住，选择人物间距较大的合照效果更佳噢！**

